准备数据

In [ ]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST Original',data_home='./')
mnist

In [ ]:
X, y = mnist["data"], mnist["target"]


In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]


In [ ]:
import numpy as np
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

开始习题

1.为MNIST数据集构建一个分类器，并在测试集上达成超过97%的精度。提示：KNeighborsClassifier对这个任务非常有效，你只需要找到合适的超参数值即可（试试对weights和n_neighbors这两个超参数进行网格搜索）。


In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.neighbors import KNeighborsClassifier

# param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [4]}]

# knn_clf = KNeighborsClassifier()
# grid_search = GridSearchCV(knn_clf, param_grid, cv=2, verbose=3, n_jobs=-1)
# grid_search.fit(X_train, y_train)

Kaggle（https://www.kaggle.com/c/titanic）上非常棒的起点：处理泰坦尼克（Titanic）数据集。

In [10]:
import pandas as pd

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [11]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


每列属性的含义
- Survived：是否幸存(0代表没有，1代表有)
- Pclass:客票级别
- Name, Sex, Age:名字，性别，年龄
- SibSp:乘客兄弟姐妹/配偶的个数(整数值)
- Parch:乘客父母/孩子的个数(整数值)
- Ticket:票号(字符串)
- Fare：乘客所持票的价格(浮点数，0-500不等)
- Cabin：乘客所在船舱(有缺失)
- Embarked：乘客登船港口:S、C、Q(有缺失)

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


好的，“Age”，“Cabin”和“Embarked”属性有时为空（小于891非空），尤其是“Cabin”（77％为空）。 我们暂时将忽略Cabin，而专注于其余部分。 Age属性大约有19％的空值，因此我们需要决定如何处理它们。 用中位数年龄替换空值似乎是合理的。


“Name”和“Ticket”属性可能有一些值，但是将它们转换为模型可以使用的有用数字会有些棘手。 所以现在，我们将忽略它们。

In [13]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Yikes, only 38% Survived. :( That's close enough to 40%, so accuracy will be a reasonable metric to evaluate our model.
The mean Fare was £32.20, which does not seem so expensive (but it was probably a lot of money back then).
The mean Age was less than 30 years old.

In [14]:
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [15]:
train_data["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [16]:
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [17]:
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

“Embarked”属性告诉我们乘客登机的地方：C =瑟堡，Q =皇后镇，S =南安普敦。

我们将重用我们在上一章中构建的DataframeSelector来从DataFrame中选择特定的属性：

In [31]:
from sklearn.base import BaseEstimator, TransformerMixin


class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.attribute_names].values

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer as SimpleImputer
num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["Age", "SibSp", "Parch", "Fare"])),
        ("imputer", SimpleImputer(strategy="median")),
    ])

In [33]:
num_pipeline.fit_transform(train_data)

array([[ 22.    ,   1.    ,   0.    ,   7.25  ],
       [ 38.    ,   1.    ,   0.    ,  71.2833],
       [ 26.    ,   0.    ,   0.    ,   7.925 ],
       ..., 
       [ 28.    ,   1.    ,   2.    ,  23.45  ],
       [ 26.    ,   0.    ,   0.    ,  30.    ],
       [ 32.    ,   0.    ,   0.    ,   7.75  ]])

对于字符串分类列，我们还将需要一个imputer（常规的`SimpleImputer`不适用于那些）：

In [34]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

警告：该书的早期版本使用LabelBinarizer或CategoricalEncoder类将每个分类值转换为一个热向量。 现在，最好使用OneHotEncoder类。 从Scikit-Learn 0.20开始，它可以处理字符串分类输入（请参阅PR＃10521），而不仅仅是整数分类输入。 如果您使用的是Scikit-Learn的旧版本，则可以从future_encoders.py导入新版本：

尝试：

In [35]:
from sklearn.base import TransformerMixin #gives fit_transform method for free
from sklearn.preprocessing import LabelBinarizer


class MyLabelBinarizer(TransformerMixin):

    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)

    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self

    def transform(self, x, y=0):
        return self.encoder.transform(x)

In [36]:

from sklearn.preprocessing import OneHotEncoder


In [37]:
cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["Pclass", "Sex", "Embarked"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder",MyLabelBinarizer),
    ])

In [38]:
cat_pipeline.fit_transform(train_data)

IndexError: arrays used as indices must be of integer (or boolean) type